In [49]:
import pandas as pd # type: ignore

In [50]:
xls = pd.ExcelFile('DATABASE.xlsx')
df1 = pd.read_excel(xls, 'PIT')
df2 = pd.read_excel(xls, 'PT_Hx')
df3 = pd.read_excel(xls, 'Para_clinic')
df4 = pd.read_excel(xls, 'Laboratory')

In [51]:
def remove_last_4_digits(x):
    return str(x)[:-6]
df1['Date'] = df1['Date'].apply(remove_last_4_digits)
df2['Date'] = df2['Date'].apply(remove_last_4_digits)
df3['Date'] = df3['Date'].apply(remove_last_4_digits)
df4['Date'] = df4['Date'].apply(remove_last_4_digits)

In [52]:
import random
import string

code_to_file_code = {}

def generate_unique_code(length):
    digits = string.digits  # '0123456789'
    return ''.join(random.choices(digits, k=length))

def fill_file_code(row):
    code = row['Code']
    if pd.isna(row['file_code']):
        if code in code_to_file_code:
            row['file_code'] = code_to_file_code[code]
        else:
            length = random.choice([6, 7])
            new_code = generate_unique_code(length)
            code_to_file_code[code] = new_code
            row['file_code'] = new_code
    return row

df1 = df1.apply(fill_file_code, axis=1)


In [53]:
merged_df = pd.DataFrame()
for code in df1['Code'].unique():
    df1_filtered = df1[df1['Code'] == code]
    df2_filtered = df2[df2['Code'] == code]
    for index, row2 in df2_filtered.iterrows():
        merged_row = pd.concat([df1_filtered.iloc[0], row2.drop('Code')])
        merged_row['Code'] = code
        merged_df = pd.concat([merged_df, merged_row.to_frame().T], ignore_index=True)
merged_df.to_excel('df.xlsx',index=True)